In [4]:
pip install datasets torch transformers

Note: you may need to restart the kernel to use updated packages.


In [8]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Trainer, TrainingArguments
import datasets
import transformers

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
cannot import name 'is_torch_xla_available' from 'transformers.utils' (C:\Users\rezag\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\utils\__init__.py)

In [4]:
# Load the Flan-T5 model and tokenizer
model_name = "google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [15]:
# Load dataset
data_files = {"train": "preprocessed_data.json", "validation": "preprocessed_data.json"}
dataset = datasets.load_dataset("json", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [16]:
# Function to tokenize the dataset
def preprocess_function(examples):
    inputs = examples["prompts"]
    targets = examples["marked_sentence"]
    if isinstance(inputs, list):
        inputs = [str(input_text) for input_text in inputs]
    else:
        inputs = [str(inputs)]
    if isinstance(targets, list):
        targets = [str(target_text) for target_text in targets]
    else:
        targets = [str(targets)]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length").input_ids
    # Replace padding token id's of the labels by -100 so they're ignored by the loss function
    labels = [[(label if label != tokenizer.pad_token_id else -100) for label in label_ids] for label_ids in labels]
    model_inputs["labels"] = labels
    return model_inputs

In [17]:
# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/4539 [00:00<?, ? examples/s]

Map:   0%|          | 0/4539 [00:00<?, ? examples/s]

In [22]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./flan_t5_event_extraction",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=2,  # Reduced batch size to prevent OOM error
    per_device_eval_batch_size=2,    # Reduced batch size to prevent OOM error
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,
    save_steps=500,
    logging_steps=100,
    fp16=torch.cuda.is_available(),
    gradient_accumulation_steps=4,  # Use gradient accumulation to simulate larger batch size
    report_to="none"  # To prevent reporting to online dashboards like Weights & Biases
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer
)

<ipython-input-23-9cd13987474d>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [25]:
# Train the model
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 5.06 MiB is free. Process 2460 has 14.74 GiB memory in use. Of the allocated memory 14.57 GiB is allocated by PyTorch, and 48.46 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Save the model
trainer.save_model("./flan_t5_event_extraction_model")

print("Model training completed and saved to './flan_t5_event_extraction_model'")